In [1]:
!pip install evaluate nltk bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.6/206.6 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.8/219.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━

In [2]:
import os
import torch
import nltk
import numpy as np
from datasets import Dataset, DatasetDict, load_from_disk# load_metric might be deprecated, use evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback
)
import evaluate # Use HF Evaluate library
import logging
import tempfile # Import tempfile


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/site-packages/torch_xla/__init__.py:251: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
E0000 00:00:1744668030.145655      10 common_lib.cc:612] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:230


In [3]:
MODEL_NAME = "t5-base"
DATASET_PATH = "/kaggle/input/endsem-eval-nlp" # <--- Use the correct Kaggle path
OUTPUT_DIR = "/kaggle/working/t5_base_counseling_finetuned" # <-- Save output to /kaggle/working/
MAX_INPUT_LENGTH = 512
MAX_TARGET_LENGTH = 128
BATCH_SIZE = 8
LEARNING_RATE = 5e-5
NUM_EPOCHS = 4
WEIGHT_DECAY = 0.01
LOGGING_STEPS = 100
EVAL_SAVE_STEPS = 400
EARLY_STOPPING_PATIENCE = 3
EARLY_STOPPING_THRESHOLD = 0.001

In [4]:
import logging

# Clear existing logging handlers (important in Jupyter)
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Now reconfigure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)

logging.info("Logging is now properly configured and working in Jupyter!")

2025-04-14 22:00:45,839 - INFO - Logging is now properly configured and working in Jupyter!


In [5]:
# *** --- START OF FIX --- ***
# Define writable cache and temp directories within /kaggle/working/
kaggle_working_dir = "/kaggle/working/"
hf_cache_dir = os.path.join(kaggle_working_dir, "hf_cache")
temp_dir = os.path.join(kaggle_working_dir, "tmp")

# Create directories if they don't exist
os.makedirs(hf_cache_dir, exist_ok=True)
os.makedirs(temp_dir, exist_ok=True)

In [6]:
# Set environment variables for Hugging Face libraries
os.environ['HF_HOME'] = hf_cache_dir
os.environ['HF_DATASETS_CACHE'] = os.path.join(hf_cache_dir, 'datasets')
os.environ['TRANSFORMERS_CACHE'] = os.path.join(hf_cache_dir, 'transformers')
os.environ['TMPDIR'] = temp_dir  # For underlying tempfile usage
os.environ['TEMP'] = temp_dir   # Windows compatibility (though not needed on Kaggle)
os.environ['TMP'] = temp_dir    # Common variant

# Explicitly set tempfile directory (optional but can help ensure consistency)
tempfile.tempdir = temp_dir

In [7]:
logging.info(f"Set HF_HOME: {os.environ.get('HF_HOME')}")
logging.info(f"Set HF_DATASETS_CACHE: {os.environ.get('HF_DATASETS_CACHE')}")
logging.info(f"Set TRANSFORMERS_CACHE: {os.environ.get('TRANSFORMERS_CACHE')}")
logging.info(f"Set TMPDIR: {os.environ.get('TMPDIR')}")
logging.info(f"Set tempfile.tempdir: {tempfile.tempdir}")
# *** --- END OF FIX --- ***

2025-04-14 22:00:45,908 - INFO - Set HF_HOME: /kaggle/working/hf_cache
2025-04-14 22:00:45,909 - INFO - Set HF_DATASETS_CACHE: /kaggle/working/hf_cache/datasets
2025-04-14 22:00:45,910 - INFO - Set TRANSFORMERS_CACHE: /kaggle/working/hf_cache/transformers
2025-04-14 22:00:45,911 - INFO - Set TMPDIR: /kaggle/working/tmp
2025-04-14 22:00:45,912 - INFO - Set tempfile.tempdir: /kaggle/working/tmp


In [8]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info(f"Using device: {device}")
if device.type == 'cuda':
    logging.info(f"GPU Name: {torch.cuda.get_device_name(0)}")

2025-04-14 22:00:45,931 - INFO - Using device: cpu


In [9]:
# --- 1. Load Dataset ---
logging.info("Loading dataset...")
try:
    train_dataset = Dataset.from_file("/kaggle/input/nlp-test/val.arrow")
    val_dataset = Dataset.from_file("/kaggle/input/nlp-test/train.arrow")
    test_dataset = Dataset.from_file("/kaggle/input/nlp-test/test.arrow")

    raw_datasets = DatasetDict({
        'train': train_dataset,
        'validation': val_dataset,
        'test': test_dataset
    })
    logging.info("Datasets loaded successfully:")
    logging.info(raw_datasets)
except Exception as e:
    logging.error(f"Error loading dataset from {DATASET_PATH}: {e}", exc_info=True)
    exit()


2025-04-14 22:00:45,948 - INFO - Loading dataset...
2025-04-14 22:00:46,011 - INFO - Datasets loaded successfully:
2025-04-14 22:00:46,013 - INFO - DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 4008
    })
    validation: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 576
    })
    test: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 968
    })
})


In [10]:
# --- 2. Load Tokenizer ---
logging.info(f"Loading tokenizer for {MODEL_NAME}...")
# (Tokenizer loading code remains the same)
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
except Exception as e:
    logging.error(f"Error loading tokenizer {MODEL_NAME}: {e}", exc_info=True)
    exit()

prefix = "generate response: "
logging.info(f"Using prefix: '{prefix}'")


# --- 3. Preprocessing ---
logging.info("Setting up preprocessing...")

2025-04-14 22:00:46,028 - INFO - Loading tokenizer for t5-base...
2025-04-14 22:00:48,174 - INFO - Using prefix: 'generate response: '
2025-04-14 22:00:48,176 - INFO - Setting up preprocessing...


In [11]:
# --- 3. Preprocessing ---
logging.info("Setting up preprocessing...")
# (preprocess_function remains the same)
def preprocess_function(examples):
    # ... (keep the function definition as before) ...
    try:
        inputs = [prefix + str(doc) for doc in examples["input_text"]] # Ensure input is string
        model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True, padding="max_length")

        # Set up the tokenizer for targets
        targets = [str(text) for text in examples["target_text"]] # Ensure target is string
        labels = tokenizer(text_target=targets, max_length=MAX_TARGET_LENGTH, truncation=True, padding="max_length")

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs
    except Exception as e:
        logging.error(f"Error during preprocessing batch: {e}", exc_info=True)
        try:
            logging.error(f"First input item: {examples.get('input_text', ['N/A'])[0]}")
            logging.error(f"First target item: {examples.get('target_text', ['N/A'])[0]}")
        except:
            logging.error("Could not retrieve specific items from batch during error.")
        raise e

logging.info("Applying preprocessing to datasets...")
try:
    # *** START OF MODIFICATION ***
    # Define explicit cache file names in the writable directory
    writable_cache_base = os.environ.get('HF_DATASETS_CACHE', '/kaggle/working/hf_cache/datasets') # Use env var if set
    os.makedirs(writable_cache_base, exist_ok=True) # Ensure base cache dir exists

    # Create a dictionary mapping split names to full cache file paths
    cache_file_names = {
        k: os.path.join(writable_cache_base, f"{k}_tokenized_{MODEL_NAME.replace('/','_')}.arrow")
        for k in raw_datasets.keys()
    }
    logging.info(f"Using explicit cache files: {cache_file_names}")

    tokenized_datasets = raw_datasets.map(
        preprocess_function,
        batched=True,
        num_proc=os.cpu_count() // 2 or 1,
        cache_file_names=cache_file_names, # <--- Explicitly provide cache file paths
        load_from_cache_file=True # Default is True, but being explicit can help
    )
    # *** END OF MODIFICATION ***

    logging.info("Preprocessing complete.")
    logging.info(tokenized_datasets)
except Exception as e:
     logging.error(f"Error during dataset mapping (preprocessing) even with explicit cache files: {e}", exc_info=True)
     # Log environment variables again to double-check
     logging.error(f"Current HF_HOME: {os.environ.get('HF_HOME')}")
     logging.error(f"Current HF_DATASETS_CACHE: {os.environ.get('HF_DATASETS_CACHE')}")
     logging.error(f"Current TMPDIR: {os.environ.get('TMPDIR')}")
     exit()


# Remove columns not needed by the model
try:
    tokenized_datasets = tokenized_datasets.remove_columns(raw_datasets["train"].column_names)
    logging.info("Removed original text columns.")
except Exception as e:
    logging.warning(f"Could not remove columns (maybe already done or preprocessing failed?): {e}")

2025-04-14 22:00:48,196 - INFO - Setting up preprocessing...
2025-04-14 22:00:48,197 - INFO - Applying preprocessing to datasets...
2025-04-14 22:00:48,199 - INFO - Using explicit cache files: {'train': '/kaggle/working/hf_cache/datasets/train_tokenized_t5-base.arrow', 'validation': '/kaggle/working/hf_cache/datasets/validation_tokenized_t5-base.arrow', 'test': '/kaggle/working/hf_cache/datasets/test_tokenized_t5-base.arrow'}
Map (num_proc=48): 100%|██████████| 968/968 [00:00<00:00, 2036.36 examples/s]
2025-04-14 22:00:54,264 - INFO - Preprocessing complete.
2025-04-14 22:00:54,266 - INFO - DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4008
    })
    validation: Dataset({
        features: ['input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 576
    })
    test: Dataset({
        features: ['input_text', 'target_text', 'input_ids', 'attention_mask', 'la

In [12]:
print("hi")

hi


In [13]:
logging.info(f"Loading model {MODEL_NAME}...")
try:
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
    model.to(device) # Move model to GPU if available
    logging.info(f"Model {MODEL_NAME} loaded successfully to {device}.")
except Exception as e:
    logging.error(f"Error loading model {MODEL_NAME}: {e}", exc_info=True)
    logging.error("Check model name, internet connection, and available memory (especially GPU RAM).")
    exit()

2025-04-14 22:00:54,340 - INFO - Loading model t5-base...
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
2025-04-14 22:00:54,635 - WARNING - Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
2025-04-14 22:00:57,817 - INFO - Model t5-base loaded successfully to cpu.


In [14]:
# --- 5. Metrics ---
logging.info("Setting up metrics (BLEU and BERTScore)...")
try:
    bleu_metric = evaluate.load("bleu")
    bertscore_metric = evaluate.load("bertscore")
except Exception as e:
    logging.error(f"Failed to load metrics using 'evaluate' library: {e}", exc_info=True)
    logging.error("Make sure 'evaluate', 'bert_score', and 'nltk' are installed: pip install evaluate bert_score nltk")
    exit()

2025-04-14 22:00:57,832 - INFO - Setting up metrics (BLEU and BERTScore)...


In [15]:
# --- 5. Metrics ---
logging.info("Setting up metrics (BLEU and BERTScore)...")
try:
    bleu_metric = evaluate.load("bleu")
    bertscore_metric = evaluate.load("bertscore")
except Exception as e:
    logging.error(f"Failed to load metrics using 'evaluate' library: {e}", exc_info=True)
    logging.error("Make sure 'evaluate', 'bert_score', and 'nltk' are installed: pip install evaluate bert_score nltk")
    exit()

2025-04-14 22:00:59,850 - INFO - Setting up metrics (BLEU and BERTScore)...


In [16]:
# NLTK is needed for BLEU tokenization
try:
    nltk.data.find('tokenizers/punkt')
    logging.info("NLTK punkt tokenizer found.")
except nltk.downloader.DownloadError:
    logging.info("Downloading nltk punkt tokenizer...")
    try:
        nltk.download('punkt', quiet=True)
        logging.info("NLTK punkt tokenizer downloaded successfully.")
    except Exception as e:
        logging.error(f"Failed to download nltk punkt: {e}", exc_info=True)
        logging.error("Check internet connection or manually download.")
        exit()


AttributeError: module 'nltk.downloader' has no attribute 'DownloadError'

In [17]:
import nltk
nltk.download('punkt')  # Download the tokenizer
nltk.data.find('tokenizers/punkt')
logging.info("NLTK punkt tokenizer found.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
2025-04-14 22:01:11,571 - INFO - NLTK punkt tokenizer found.


In [18]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    if isinstance(predictions, tuple): # Sometimes Trainer wraps predictions
        predictions = predictions[0]

    # Decode generated tokens into text
    # Replace -100 (ignore index) with pad_token_id for decoding
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    try:
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    except Exception as e:
        logging.error(f"Error during token decoding in compute_metrics: {e}", exc_info=True)
        # Log shapes to help debug
        logging.error(f"Predictions shape: {predictions.shape if isinstance(predictions, np.ndarray) else 'N/A'}")
        logging.error(f"Labels shape: {labels.shape if isinstance(labels, np.ndarray) else 'N/A'}")
        # Return dummy metrics to avoid crashing training, but log the error clearly
        return {"bleu": 0.0, "bertscore_f1": 0.0}


    # Simple post-processing: remove leading/trailing spaces
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]
    # Handle empty predictions/labels which can cause errors in metrics
    decoded_preds = [pred if pred else "<empty>" for pred in decoded_preds]
    decoded_labels = [label if label else "<empty>" for label in decoded_labels]

    # Compute BLEU
    try:
        tokenized_preds_for_bleu = [nltk.word_tokenize(pred.lower()) for pred in decoded_preds]
        tokenized_labels_for_bleu = [[nltk.word_tokenize(label.lower())] for label in decoded_labels] # BLEU expects list of references
        bleu_result = bleu_metric.compute(predictions=tokenized_preds_for_bleu, references=tokenized_labels_for_bleu)
        if bleu_result is None or "bleu" not in bleu_result:
            logging.warning("BLEU computation returned None or unexpected format. Setting BLEU to 0.")
            bleu_score = 0.0
        else:
            bleu_score = bleu_result["bleu"]
    except Exception as e:
        logging.error(f"Error computing BLEU score: {e}", exc_info=True)
        bleu_score = 0.0 # Assign default value on error

    # Compute BERTScore
    try:
        # Specify language for BERTScore model (important)
        # Also specify the model type if default isn't optimal (check BERTScore docs)
        bertscore_result = bertscore_metric.compute(predictions=decoded_preds, references=decoded_labels, lang="en", device=device) # Run on same device as model if possible
        if bertscore_result is None or "f1" not in bertscore_result or not bertscore_result["f1"]:
            logging.warning("BERTScore computation returned None, empty F1 list, or unexpected format. Setting BERTScore F1 to 0.")
            bertscore_f1 = 0.0
            bertscore_precision = 0.0
            bertscore_recall = 0.0
        else:
            bertscore_f1 = np.mean(bertscore_result["f1"])
            bertscore_precision = np.mean(bertscore_result["precision"])
            bertscore_recall = np.mean(bertscore_result["recall"])
    except Exception as e:
        logging.error(f"Error computing BERTScore: {e}", exc_info=True)
        # Check for common issues like OOM on BERTScore model if running on GPU
        if "CUDA out of memory" in str(e):
            logging.warning("BERTScore calculation failed due to OOM. Consider running on CPU or reducing batch size.")
        bertscore_f1 = 0.0 # Assign default value on error
        bertscore_precision = 0.0
        bertscore_recall = 0.0

    # Combine results
    result = {"bleu": bleu_score}
    result["bertscore_f1"] = bertscore_f1
    result["bertscore_precision"] = bertscore_precision
    result["bertscore_recall"] = bertscore_recall


    # Add generation length metric (useful for debugging)
    try:
        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
        result["gen_len"] = np.mean(prediction_lens)
    except Exception as e:
        logging.warning(f"Could not compute gen_len: {e}")
        result["gen_len"] = 0.0

    return {k: round(v, 4) for k, v in result.items()} # Round metrics

In [19]:
# --- 6. Training Arguments ---
logging.info("Configuring training arguments...")
training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="steps",
    eval_steps=EVAL_SAVE_STEPS,
    save_strategy="steps",
    save_steps=EVAL_SAVE_STEPS,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE * 2, # Usually can use larger batch for eval
    weight_decay=WEIGHT_DECAY,
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,      # VERY Important for Seq2Seq metrics
    logging_dir=f"{OUTPUT_DIR}/logs",
    logging_strategy="steps",
    logging_steps=LOGGING_STEPS,
    load_best_model_at_end=True,     # Load best model based on metric_for_best_model
    metric_for_best_model="bertscore_f1", # Optimize for BERTScore F1 (often better aligns with semantics)
    greater_is_better=True,
    # push_to_hub=False,             # Set to True to upload to Hugging Face Hub
    fp16=torch.cuda.is_available(),  # Enable mixed precision training if GPU supports it (speeds up, saves memory)
    # report_to="tensorboard",       # Enable TensorBoard logging
    generation_max_length=MAX_TARGET_LENGTH, # Ensure generated sequences aren't too long during eval
    save_total_limit=3,              # Keep only the best 3 checkpoints
)

# --- 7. Data Collator ---
# Pads inputs and labels dynamically
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

2025-04-14 22:01:16,536 - INFO - Configuring training arguments...
E0000 00:00:1744668078.587483      10 common_lib.cc:621] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:232
2025-04-14 22:01:22,283 - WARNING - torch_xla.core.xla_model.xrt_world_size() will be removed in release 2.7. is deprecated. Use torch_xla.runtime.world_size instead.
2025-04-14 22:01:22,287 - WARNING - torch_xla.core.xla_model.xla_model.get_ordinal() will be removed in release 2.7. is deprecated. Use torch_xla.runtime.global_ordinal instead.


In [20]:
# --- 8. Trainer ---
logging.info("Initializing Trainer...")
# Add EarlyStopping callback
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=EARLY_STOPPING_PATIENCE,
    early_stopping_threshold=EARLY_STOPPING_THRESHOLD,
)

2025-04-14 22:01:22,337 - INFO - Initializing Trainer...


In [21]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping] # Add the callback here
)

/kaggle/working/tmp/ipykernel_10/529137534.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
# --- 9. Train ---
logging.info("Starting training...")
try:
    train_result = trainer.train()
    logging.info("Training finished.")

    # Save training metrics and state
    trainer.save_model() # Saves the tokenizer too
    logging.info(f"Best model saved to {OUTPUT_DIR}")

    metrics = train_result.metrics
    metrics["train_samples"] = len(tokenized_datasets["train"])
    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()
    logging.info(f"Training metrics and state saved.")

except Exception as e:
    logging.error(f"Error during training: {e}", exc_info=True)
    # Save potentially partially trained model if needed
    # trainer.save_model(os.path.join(OUTPUT_DIR, "interrupted_model"))
    # logging.info("Attempted to save interrupted model.")
    exit()


2025-04-14 22:01:28,289 - INFO - Starting training...
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss


In [ ]:
# --- 10. Evaluate on Test Set ---
logging.info("Evaluating on the test set using the best model...")
try:
    # Ensure the best model is loaded if load_best_model_at_end=True
    # (Trainer does this automatically before evaluate if load_best_model_at_end=True)
    test_metrics = trainer.evaluate(eval_dataset=tokenized_datasets["test"], metric_key_prefix="test")

    metrics = test_metrics
    metrics["test_samples"] = len(tokenized_datasets["test"])
    trainer.log_metrics("test", metrics)
    trainer.save_metrics("test", metrics)
    logging.info("Test set evaluation complete.")
    logging.info(f"Test Metrics: {test_metrics}")

except Exception as e:
    logging.error(f"Error during test set evaluation: {e}", exc_info=True)
    # You might still want to try inference below even if evaluation failed

# --- 11. Example Inference ---
logging.info("\n--- Example Inference ---")
try:
    # Use the trainer's model (which should be the best one)
    # Or explicitly load if needed:
    # logging.info(f"Loading best model from {OUTPUT_DIR} for inference...")
    # model = AutoModelForSeq2SeqLM.from_pretrained(OUTPUT_DIR).to(device)
    # tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR)

    if len(raw_datasets["test"]) > 0:
        test_example_input = raw_datasets["test"][0]["input_text"]
        test_example_target = raw_datasets["test"][0]["target_text"]

        logging.info(f"Input Text:\n{test_example_input}")
        logging.info(f"\nTarget Text:\n{test_example_target}")

        # Prepare input for the model
        input_text_with_prefix = prefix + test_example_input
        inputs = tokenizer(input_text_with_prefix, return_tensors="pt", max_length=MAX_INPUT_LENGTH, truncation=True, padding=True).to(device)

        # Generate output - Experiment with these parameters!
        output_sequences = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=MAX_TARGET_LENGTH + 10, # Allow slightly longer generation than target max
            num_beams=5,                       # Beam search often improves coherence and relevance (try 3-10)
            early_stopping=True,               # Stop generation when beams converge
            # --- Parameters for Novelty/Diversity ---
            # temperature=0.8,                 # >1.0 more random, <1.0 more focused. Default is 1.0. Try 0.7-0.9?
            # top_k=50,                        # Sample from top K most likely tokens. Reduces gibberish. (e.g., 50)
            # top_p=0.95,                      # Nucleus sampling: sample from smallest set whose prob >= p. (e.g., 0.9-0.95)
            # no_repeat_ngram_size=3,          # Prevent repeating the same 3-grams. Helps reduce repetition.
            # num_return_sequences=1,          # Generate multiple sequences if needed (e.g., for evaluation or choice)
        )

        # Decode the generated sequence(s)
        generated_texts = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)

        logging.info("\n--- Generated Text ---")
        for i, text in enumerate(generated_texts):
            logging.info(f"Output {i+1}: {text}")

    else:
        logging.warning("Test dataset is empty, cannot perform example inference.")

except Exception as e:
    logging.error(f"Error during example inference: {e}", exc_info=True)

logging.info("\nScript finished.")